# Day 17

In [69]:
with open('../inputs/adventofcode.com_2022_day_17_input.txt', 'r') as f:
    data = f.read().splitlines()
data = data[0]
print(f'The file contains {len(data)} movements.')

The file contains 10091 movements.


## Puzzle 1

In [70]:
import numpy as np

tunnel = np.zeros((1, 7), dtype=int)

num_rocks = 2022

move = 0

rocks = {}

rocks[0] = np.ones((1, 4), dtype=int)
rocks[1] = np.array([[0,1,0], [1,1,1], [0,1,0]], dtype=int)
rocks[2] = np.array([[0,0,1], [0,0,1], [1,1,1]], dtype=int)
rocks[3] = np.ones((4, 1), dtype=int)
rocks[4] = np.ones((2, 2), dtype=int)

def movement_possible(tunnel, coords, rock_type):
    # For each of the cells occupied by the rock, check if the tunnel is empty
    for row in range(rocks[rock_type].shape[0]):
        for col in range(rocks[rock_type].shape[1]):
            if rocks[rock_type][row, col] == 1 and tunnel[coords[0]+row, coords[1]+col] == 1:
                return False
    return True
        

for rock in range(num_rocks):
    rock_type = rock % 5
    highest_rock = np.argmax(np.sum(tunnel, axis=1)!=0)

    # If highest rock is 0, then the tunnel is empty
    if highest_rock == 0:
        highest_rock = 1
    
    # Top coordinate of the rock
    top = highest_rock - 3 - rocks[rock_type].shape[0]

    # If the rock is too high, add empty rows at the top
    if top < 0:
        tunnel = np.vstack((np.zeros((-top, tunnel.shape[1]), dtype=int), tunnel))
        highest_rock += -top
        coords = (0, 2)
    else:
        coords = (top, 2)

    while True:
        # Move left or right
        if data[move % len(data)] == '>' and coords[1] + rocks[rock_type].shape[1] < tunnel.shape[1] and movement_possible(tunnel, (coords[0], coords[1]+1), rock_type):
            coords = (coords[0], coords[1]+1)
        elif data[move % len(data)] == '<' and coords[1] > 0 and movement_possible(tunnel, (coords[0], coords[1]-1), rock_type):
            coords = (coords[0], coords[1]-1)
        move += 1
        
        # Move down
        if coords[0]+1 + rocks[rock_type].shape[0] <= tunnel.shape[0] and movement_possible(tunnel, (coords[0]+1, coords[1]), rock_type):
            coords = (coords[0]+1, coords[1])
        else:
            break
    # With the final coordinates, add the rock to the tunnel
    tunnel[coords[0]:coords[0]+rocks[rock_type].shape[0], coords[1]:coords[1]+rocks[rock_type].shape[1]] += rocks[rock_type]
    
height = tunnel.shape[0] - np.argmax(np.sum(tunnel, axis=1)!=0)
print(f'After {num_rocks} the tower is {height} units high.')

After 2022 the tower is 3161 units high.
